<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/knlp_rnn_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#참고:  텐서플로와 머신러닝으로 시작하는 자연어 처리
!pip install kaggle
from google.colab import files
files.upload()

In [2]:
import os
os.chdir('/content')
!ls -1ha kaggle.json

kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp.git
import os
os.chdir('/content/tensorflow-ml-nlp')
!pip install -r requirements.txt


Cloning into 'tensorflow-ml-nlp'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 845 (delta 32), reused 27 (delta 15), pack-reused 778
Receiving objects: 100% (845/845), 160.14 MiB | 26.27 MiB/s, done.
Resolving deltas: 100% (499/499), done.
Checking out files: 100% (100/100), done.
     |████████████████████████████████| 19.4MB 4.7MB/s 
     |████████████████████████████████| 174kB 46.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/0e/2b/e8/c0b818ac4b3d35104d35e48cdc7afe27fc06ea277feed2831a
Successfully built JPype1


In [5]:
!mkdir -p /content/kaggle_bag_of_word
os.chdir('/content/kaggle_bag_of_word')
!kaggle competitions download -c word2vec-nlp-tutorial

  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 37.8MB/s]
 35% 9.00M/26.0M [00:00<00:00, 20.8MB/s]
100% 26.0M/26.0M [00:00<00:00, 44.3MB/s]
 95% 12.0M/12.6M [00:00<00:00, 39.7MB/s]
100% 12.6M/12.6M [00:00<00:00, 50.1MB/s]
 69% 9.00M/13.0M [00:00<00:00, 35.9MB/s]
100% 13.0M/13.0M [00:00<00:00, 43.2MB/s]


In [0]:
import zipfile

DATA_IN_PATH = '/content/kaggle_bag_of_word/'

file_list = ['labeledTrainData.tsv.zip','unlabeledTrainData.tsv.zip', 'testData.tsv.zip']

for file in file_list:
  zipRef = zipfile.ZipFile(DATA_IN_PATH + file, 'r')
  zipRef.extractall(DATA_IN_PATH)
  zipRef.close()
             

In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline # 그래프를 주피터 노트북에서 바로 그리도록 함

UsageError: unrecognized arguments: # 그래프를 주피터 노트북에서 바로 그리도록 함


In [0]:
train_data = pd.read_csv(DATA_IN_PATH+"labeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)

In [0]:
train_word_counts = train_data['review'].apply(lambda x:len(x.split(' ')))

In [0]:
import re
import pandas as pd
import numpy
import json
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
DATA_IN_PATH = '/content/kaggle_bag_of_word/'
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv',header = 0, delimiter = '\t', quoting = 3)

In [0]:
def preprocessing(review, remove_stopwords = False):
  review_text = BeautifulSoup(review, "html5lib").get_text()
  review_text = re.sub("[^a-zA-Z]", " ", review_text)
  words = review_text.lower().split()
  if remove_stopwords:
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    clean_review = " ".join(words)
  else:
    clean_review = " ".join(words)
    
  return clean_review

In [0]:
clean_train_reviews = []
for review in train_data['review']:
  clean_train_reviews.append(preprocessing(review, remove_stopwords = True))
  
clean_train_df = pd.DataFrame({'review':clean_train_reviews, 'sentiment': train_data['sentiment']})

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

word_vocab = tokenizer.word_index

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')
train_labels = np.array(train_data['sentiment'])

In [0]:
DATA_IN_PATH = '/content/kaggle_bag_of_word/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS = 'data_configs.json'

import os

if not os.path.exists(DATA_IN_PATH):
  os.makedir(DATA_IN_PATH)

np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

clean_train_df.to_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, index = False)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)

In [0]:
test_data = pd.read_csv(DATA_IN_PATH + "testData.tsv", header = 0, delimiter = "\t", quoting = 3)

clean_test_reviews = []

for review in test_data['review']:
  clean_test_reviews.append(preprocessing(review, remove_stopwords = True))
clean_test_df =  pd.DataFrame({'review': clean_test_reviews, 'id': test_data['id']})
test_id = np.array(test_data['id'])

tokenizer.fit_on_texts(clean_test_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_test_reviews)
test_inputs = pad_sequences(text_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')

In [0]:
DATA_IN_PATH = '/content/kaggle_bag_of_word/'
DATA_OUT_PATH = '/content/kaggle_bag_of_word/data_out/'

INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

train_input = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
train_label = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))

prepro_configs = None

with open(DATA_IN_PATH + DATA_CONFIGS_FILE_NAME, 'r') as f:
  prepro_configs = json.load(f)

In [0]:
from sklearn.model_selection import train_test_split

TEST_SPLIT = 0.2
RANDOM_SEED = 13371447

input_train, input_eval, label_train, lavel_eval = train_test_split(train_input,
                                                                    train_label,
                                                                    test_size = TEST_SPLIT,
                                                                    random_state = RANDOM_SEED)

In [0]:
import tensorflow as tf

BATCH_SIZE = 16
NUM_EPOCHS = 3

def mapping_fn(X, Y):
  inputs, labels = {'x' : X}, Y
  return inputs, labels

def train_input_fn():
  dataset = tf.data.Dataset.from_tensor_slice((input_train, label_train))
  dataset = dataset.shuffle(buffer_size = 1000)
  dataset = dataset.map(mapping_fn)
  dataset = dataset.repeat(count = NUM_EPOCHS)
  iterator = dataset.make_one_shot_iterator()
  
  return iterator.get_next()

def eval_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((input_test, label_test))
  dataset = dataset.map(mapping_fn)
  dataset = dataset.batch(BATCH_SIZE)
  iterator = dataset.make_one_shot_iterator()
  
  return iterator.get_next()

In [0]:
VOCAB_SIZE = prepro_configs['vocab_size']

In [0]:
WORD_EMBEDDING_DIM = 1000
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150

learning_rate = 0.001

In [0]:
def model_fn(features, labels, mode):
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN
  EVAL = mode == tf.estimator.ModeKeys.EVAL
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT
  
  embedding_layer = tf.keras.layers.Embedding(VOCAB_SIZE,
                                             WORD_EMBEDDING_DIM)(features['x'])
  
  embedding_layer = tf.keras.layers.Dropout(0.2)(embedding_layer)
  
  rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
  multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
  
  outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,inputs = embedding_layer, dtype = tf.float32)
  hidden_layer = tf.keras.layers.Dense(DENSE_FEATURE_DIM, activation = tf.nn.tanh)(outputs[:,-1,:])
  hidden_layer = tf.keras.layers.Dropout(0.2)(hidden_layer)
  logits = tf.keras.layers.Dense(1)(hidden_layer)
  logits = tf.squeeze(logits, axis = -1)
  
  sigmoid_logits = tf.nn.sigmoid(logits)
  
  if PREDICT:
    predictions = {'sentiment': sigmoid_logits}
    
    return tf.estimator.EstimatorSpec(mode = mode,
                                      predictions = predictions)
  
  loss = tf.losses.sigmoid_cross_entropy(labels, logits)
  
  if EVAL:
    accuracy = tf.metrics.accuracy(labels, tf.round(sigmoid_logits))
    eval_metric_ops = {'acc': accuracy}
    
    return tf.estimator.EstimatorSpec(mode, loss = loss,
                                      eval_metric_ops = eval_metric_ops)
  
  if TRAIN:
    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
    
    return tf.estimator.EsimatorSpec(mode = mode,
                                     train_op = train_op,
                                     loss = loss)

In [24]:
accuracy = tf.metrics.accuracy(labels, tf.round(sigmoid_logits))
eval_metric_ops = {'acc':accuracy}

return tf.estimator.EstimatorSpec(mode, loss = loss, eval_metric_ops = eval_metric_ops)

NameError: ignored

In [0]:
np.zeros((5,5))